## Daymet

### Resources

- Working with netCDF files: https://towardsdatascience.com/read-netcdf-data-with-python-901f7ff61648
- NetCDF4 documentation: https://unidata.github.io/netcdf4-python/
- Daymet data description: https://daac.ornl.gov/DAYMET/guides/Daymet_V4_Stn_Level_CrossVal.html


### Data Description:
- Daily weather observation data and cross-validation results for three Daymet model parameters: min temp, max temp, and daily precipitation
- Observations are from multiple stations within the same region.
- The North American region includes Canada, United States, and Mexico. The data for the North American region spans 1980-present.
- Also included are corresponding station metadata files listing every surface weather station used in Daymet processing for each parameter, region, and year and containing the station name, station identification, latitude, and longitude.
- There are 468 netCDF files that contain the daily input observations and cross-validation results. Each variable-region-year combination is a separate file. 
- There are 126 files each for North America and Hawaii, and 216 files for Puerto Rico. 
- Provided as additional files, are 468 corresponding text files (.txt) that contain metadata for every surface weather station used in Daymet processing for the variable-region-year combinations. 
- Filename format: daymet_v4_stnxval_pppp_region_yyyy.nc, where
    - pppp: is the respective meteorological variable (tmax, tmin, and prcp)
    - region: is 'na' (for continental North America), 'hi' (for Hawaii), or 'pr' (for Puerto Rico), and
    - yyyy: is year (1950 through 2021).  
- Parameters in the netCDF file
    - station_id: The ground weather station identification (as a string value) as extracted from the GHCN-Daily input files.
    - station_name: The ground weather station name (as a string value) as extracted from the GHCN-Daily input files.
    - stnz (meters): The station elevation reported in the metadata readme as extracted from the GHCN-Daily input files.
    - time (day): The day number since the beginning of the dataset. Data are in a daily time step, defined as 24-hour day based on local time from mid-night to mid-night.
    - time_bnds (day): The start and end time points for each day (24-hr period based on local time).
    - stns: An integer station index within the netCDF file.
    - stn_lon (decimal degrees): The station longitude in WGS 84 (EPSG:4326) reported in the metadata readme as extracted from the GHCN-Daily input files.
    - stn_lat (decimal degrees): The station latitude in WGS 84 (EPSG:4326) reported in the metadata readme as extracted from the GHCN-Daily input files.
- Parameters specific to the precipitation netCDF file
    - obs (mm): Station observed daily total precipitation in millimeters, sum of all forms converted to water-equivalent. Data are in a daily time step, defined as 24-hour day based on local time from mid-night to mid-night.
    - pred (mm): Daymet model predicted daily total precipitation in millimeters, using Daymet’s cross-validation protocol, sum of all forms converted to water-equivalent. Data are in a daily time step, defined as 24-hour day based on local time from mid-night to mid-night.
- Parameters specific to maximum and minimum temperature netCDF files
    - obs (degrees C): Station observed daily maximum (or minimum respective of file) 2-meter air temperature in degrees Celsius.  Data are in a daily time step, defined as 24-hour day based on local time from mid-night to mid-night.
    - pred (degrees C): Daymet model predicted daily maximum (or minimum respective of file) 2 m air temperature in degrees Celsius using Daymet’s cross-validation protocol.  Data are in a daily time step, defined as 24-hour day based on local time from mid-night to mid-night.

In [1]:
import pandas as pd
import numpy as np
import numpy.ma as ma
import netCDF4 as nc
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
fn = 'Daymet_V4_Stn_Level_CrossVal_1850/data/daymet_v4_stnxval_prcp_hi_1980.nc'

In [57]:
data = nc.Dataset(fn, 'r', format='NETCDF4_CLASSIC')
data.variables.keys()

dict_keys(['pred', 'obs', 'stns', 'station_id', 'station_name', 'stnx', 'stny', 'stnz', 'time', 'time_bnds', 'stn_lon', 'stn_lat'])

In [4]:
variables = ['station_id', 'station_name', 'stnz', 'time', 'time_bnds', 'stns', 'stn_lon', 'stn_lat', 'pred', 'obs']

for var in variables:
    print(data.variables[var].shape)

(154, 255)
(154, 255)
(154,)
(365,)
(365, 2)
(154,)
(154,)
(154,)
(154, 365)
(154, 365)


In [5]:
def get_station_ids(colname='station_id'):
    arr = data.variables[colname][:]
    stations = list()
    for v, m in zip(ma.getdata(arr), ma.getmask(arr)):
        station = ''.join(v[~m].astype('U13').tolist())
        stations.append(station)
    return stations

def get_station_names(colname='station_name'):
    arr = data.variables[colname][:]
    stations = list()
    for v in ma.getdata(arr):
        station = ''.join(v.astype('U13').tolist()).strip()
        stations.append(station)
    return stations

In [89]:
# station information
def get_station_df():
    station_ids = get_station_ids(colname='station_id')
    station_names = get_station_names(colname='station_name')
    station_elevation = ma.getdata(data.variables['stnz'][:])
    station_lon = ma.getdata(data.variables['stn_lon'][:])
    station_lat = ma.getdata(data.variables['stn_lat'][:])
    station_int_index = ma.getdata(data.variables['stns'][:])

    for i in [station_ids, station_names, station_elevation, station_int_index]:
        print(f'number of rows: {len(i)}')

    station_df = pd.DataFrame()
    station_df['station_id'] = station_ids
    station_df['station_name'] = station_names
    station_df['stn_lon'] = station_lon
    station_df['stn_lat'] = station_lat
    station_df['stnz'] = station_elevation
    station_df['key'] = 1
    
    return station_df

In [90]:
# time data
def get_time_df():
    time = ma.getdata(data.variables['time'][:])
    time_bnds = ma.getdata(data.variables['time_bnds'][:])


    time_df = pd.DataFrame()
    time_df['time'] = time
    time_df['time_bnds_lower'] = [i[0] for i in time_bnds]
    time_df['time_bnds_upper'] = [i[1] for i in time_bnds]
    time_df['key'] = 1
    return time_df

In [91]:
station_df = get_station_df()
time_df = get_time_df()

number of rows: 154
number of rows: 154
number of rows: 154
number of rows: 154


In [103]:
print(station_df.head(3))
print(time_df.head(3))

    station_id           station_name  stn_lon  stn_lat   stnz  key
0  USC00510145           ANAHOLA 1114 -159.304  22.1322   54.9    1
1  USC00510935           HALAULA 1110 -159.317  22.1164   77.1    1
2  USC00512222  ILIILIULA INTAKE 1050 -159.467  22.0333  320.0    1
      time  time_bnds_lower  time_bnds_upper  key
0  10957.5          10957.0          10958.0    1
1  10958.5          10958.0          10959.0    1
2  10959.5          10959.0          10960.0    1


In [98]:
station_time_df = pd.merge(station_df, time_df, on ='key').drop("key", axis=1)

In [102]:
station_time_df

,station_id,station_name,stn_lon,stn_lat,stnz,time,time_bnds_lower,time_bnds_upper
0,USC00510145,ANAHOLA 1114,-159.304,22.1322,54.9,10957.5,10957.0,10958.0
1,USC00510145,ANAHOLA 1114,-159.304,22.1322,54.9,10958.5,10958.0,10959.0
2,USC00510145,ANAHOLA 1114,-159.304,22.1322,54.9,10959.5,10959.0,10960.0
3,USC00510145,ANAHOLA 1114,-159.304,22.1322,54.9,10960.5,10960.0,10961.0
4,USC00510145,ANAHOLA 1114,-159.304,22.1322,54.9,10961.5,10961.0,10962.0
...,...,...,...,...,...,...,...,...
56205,USC00518830,UPOLU POINT USCG 159.2,-155.883,20.2500,18.6,11317.5,11317.0,11318.0
56206,USC00518830,UPOLU POINT USCG 159.2,-155.883,20.2500,18.6,11318.5,11318.0,11319.0
56207,USC00518830,UPOLU POINT USCG 159.2,-155.883,20.2500,18.6,11319.5,11319.0,11320.0
56208,USC00518830,UPOLU POINT USCG 159.2,-155.883,20.2500,18.6,11320.5,11320.0,11321.0


In [152]:
# obs data
pred = ma.getdata(data.variables['pred'][:])
obs = ma.getdata(data.variables['obs'][:])

In [157]:
pred_df = pd.DataFrame(pred.T, columns=station_df['station_id'])
obs_df = pd.DataFrame(pred.T, columns=station_df['station_id'])

pred_df.columns.name = None
obs_df.columns.name = None
# obs['key'] = 1

In [164]:
pred_obs_df = pd.merge(pred_df, obs_df, left_index=True, right_index=True, suffixes=('_pred', '_obs'))

In [165]:
pd.merge(time_df, pred_obs_df, left_index=True, right_index=True)#.drop('key', axis=1)

,time,time_bnds_lower,time_bnds_upper,key,USC00510145_pred,USC00510935_pred,USC00512222_pred,USC00513099_pred,USC00513104_pred,USC00513982_pred,...,USC00511339_obs,USC00514680_obs,USC00514815_obs,USC00514928_obs,USC00514938_obs,USC00517131_obs,USC00517204_obs,USC00517312_obs,USC00518422_obs,USC00518830_obs
0,10957.5,10957.0,10958.0,1,-9999.0,-9999.0,-9999.0,-9999.000000,-9999.0,-9999.0,...,6.447345,10.188461,4.297348,3.079690,7.286470,0.000000,3.794753,3.924101,0.0,0.000000
1,10958.5,10958.0,10959.0,1,-9999.0,-9999.0,-9999.0,0.000000,-9999.0,-9999.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,10959.5,10959.0,10960.0,1,-9999.0,-9999.0,-9999.0,0.000000,-9999.0,-9999.0,...,0.929150,0.000000,2.644703,3.080913,0.000000,0.000000,0.071905,0.000000,0.0,0.000000
3,10960.5,10960.0,10961.0,1,-9999.0,-9999.0,-9999.0,0.000000,-9999.0,-9999.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,10961.5,10961.0,10962.0,1,-9999.0,-9999.0,-9999.0,3.569688,-9999.0,-9999.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,11317.5,11317.0,11318.0,1,-9999.0,-9999.0,-9999.0,0.000000,-9999.0,-9999.0,...,0.000000,0.000000,7.152267,8.166018,7.664408,11.245600,8.777375,10.101097,0.0,9.056848
361,11318.5,11318.0,11319.0,1,-9999.0,-9999.0,-9999.0,0.000000,-9999.0,-9999.0,...,2.271241,-9999.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,2.920197
362,11319.5,11319.0,11320.0,1,-9999.0,-9999.0,-9999.0,0.000000,-9999.0,-9999.0,...,1.827755,-9999.000000,1.517328,2.224664,1.509852,1.605328,1.314377,2.246907,0.0,0.000000
363,11320.5,11320.0,11321.0,1,0.0,0.0,-9999.0,2.389517,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [161]:
pred_obs_df

,USC00510145_x,USC00510935_x,USC00512222_x,USC00513099_x,USC00513104_x,USC00513982_x,USC00514272_x,USC00514561_x,USC00514568_x,USC00514735_x,...,USC00511339_y,USC00514680_y,USC00514815_y,USC00514928_y,USC00514938_y,USC00517131_y,USC00517204_y,USC00517312_y,USC00518422_y,USC00518830_y
0,-9999.0,-9999.0,-9999.0,-9999.000000,-9999.0,-9999.0,0.000000,0.000000,0.000000,0.000000,...,6.447345,10.188461,4.297348,3.079690,7.286470,0.000000,3.794753,3.924101,0.0,0.000000
1,-9999.0,-9999.0,-9999.0,0.000000,-9999.0,-9999.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,-9999.0,-9999.0,-9999.0,0.000000,-9999.0,-9999.0,0.000000,0.000000,0.000000,0.000000,...,0.929150,0.000000,2.644703,3.080913,0.000000,0.000000,0.071905,0.000000,0.0,0.000000
3,-9999.0,-9999.0,-9999.0,0.000000,-9999.0,-9999.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,-9999.0,-9999.0,-9999.0,3.569688,-9999.0,-9999.0,2.564445,7.223607,9.342837,1.078652,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,-9999.0,-9999.0,-9999.0,0.000000,-9999.0,-9999.0,0.000000,0.000000,-9999.000000,0.000000,...,0.000000,0.000000,7.152267,8.166018,7.664408,11.245600,8.777375,10.101097,0.0,9.056848
361,-9999.0,-9999.0,-9999.0,0.000000,-9999.0,-9999.0,0.000000,0.000000,-9999.000000,0.000000,...,2.271241,-9999.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,2.920197
362,-9999.0,-9999.0,-9999.0,0.000000,-9999.0,-9999.0,0.000000,2.708354,-9999.000000,0.000000,...,1.827755,-9999.000000,1.517328,2.224664,1.509852,1.605328,1.314377,2.246907,0.0,0.000000
363,0.0,0.0,-9999.0,2.389517,0.0,0.0,0.310080,0.000000,0.000000,2.705069,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
